In [ ]:
import os, requests
import glob
import numpy as np
import re
import struct
# 既存のTellus環境にgeocoderは入ってないので、!pip install geocoderでインストールしてください
import geocoder

# Fields
BASE_API_URL = "https://file.tellusxdp.com/api/v1/origin/search/" # https://www.tellusxdp.com/docs/api-reference/palsar2-files-v1.html#/
ACCESS_TOKEN = "*******************" # Tellus登録時のアクセストークンを入れてください
HEADERS = {"Authorization": "Bearer " + ACCESS_TOKEN}
TARGET_PLACE="tokyo" # ここに取得した地名を入れてください
SAVE_DIRECTORY="./SAR_{}/".format(TARGET_PLACE)

# Functions
def rect_vs_point(ax, ay, aw, ah, bx, by):
    return 1 if bx > ax and bx < aw and by > ay and by < ah else 0
def get_scene_list_slc(_get_params={}):
    query = "palsar2-l11"
    r = requests.get(BASE_API_URL + query, _get_params, headers=HEADERS)
    if not r.status_code == requests.codes.ok:
        r.raise_for_status()
    return r.json()

def get_slc_scenes(_target_json, _get_params={}):
    # get file list
    r = requests.get(_target_json["publish_link"], _get_params, headers=HEADERS)
    if not r.status_code == requests.codes.ok:
        r.raise_for_status()
    file_list = r.json()['files']
    dataset_id = _target_json['dataset_id'] # folder name
    # make dir
    if os.path.exists(SAVE_DIRECTORY + dataset_id) == False:
        os.makedirs(SAVE_DIRECTORY + dataset_id)
    # downloading
    print("[Start downloading]", dataset_id)
    for _tmp in file_list:
        r = requests.get(_tmp['url'], headers=HEADERS, stream=True)
        if not r.status_code == requests.codes.ok:
            r.raise_for_status()
        with open(os.path.join(SAVE_DIRECTORY, dataset_id, _tmp['file_name']), "wb") as f:
            f.write(r.content)
        print("  - [Done]", _tmp['file_name'])
    print("finished") 
    return

# Entry point
def main():
    # extract slc list around the address
    gc = geocoder.osm(TARGET_PLACE, timeout=5.0) # get latlon
    slc_list_json = get_scene_list_slc({'after':'2019-1-01T00:00:00', 'polarisations':'HH+HV+VV+VH', 'page_size':'1000'})  # get single look complex list
    target_places_json = [_ for _ in slc_list_json['items'] if rect_vs_point(_['bbox'][1], _['bbox'][0], _['bbox'][3], _['bbox'][2], gc.latlng[0], gc.latlng[1])] # lot_min, lat_min, lot_max...
    target_ids = [_['dataset_id'] for _ in target_places_json]
    print("[Matched SLCs]", target_ids)
    for target_id in target_ids:
        target_json = [_ for _ in slc_list_json['items'] if _['dataset_id'] == target_id][0]
        # download the target file
        get_slc_scenes(target_json)
        #break # if remove, download all data
    
if __name__=="__main__":
       main()

In [ ]:
# 上記の結果を参考にして、今回は以下の二つのSARデータを取り出すことにしました。
target_id1 = 'SAR_tokyo/ALOS2267472890-190507'
target_id2 = 'SAR_tokyo/ALOS2250912890-190115'

In [ ]:
class Get_image():
    def __init__(self, dataset_id=''):
        self.dataset_id = dataset_id
    
    def SAR_data(self):
        names = ['HH','HV','VV']
        lists = glob.glob(self.dataset_id + '/*')
        CEOS_lists = sorted({k for k in lists if re.search('IMG-(HH|HV|VV)', str(k))})
        i=0
        for filename in CEOS_lists:
            fp = open(os.path.join('/home','jovyan','work',filename),mode='rb')
            col = 2000
            row = 2000
            cr = np.array([3000,3000+col,8000,8000+row],dtype="i4");
            
            fp.seek(236)
            nline = int(fp.read(8))
            fp.seek(248)
            ncell = int(fp.read(8))
            nrec = 544 + ncell*8

            nline = cr[3]-cr[2]
            fp.seek(720)
            fp.read(int((nrec/4)*(cr[2])*4))
            data = struct.unpack(">%s"%(int((nrec*nline)/4))+"f",fp.read(int(nrec*nline)))
            data = np.array(data).reshape(-1,int(nrec/4))
            data = data[:,int(544/4):int(nrec/4)]
            slc = data[:,::2] + 1j*data[:,1::2]
            slc = slc[:,cr[0]:cr[1]]
            print(np.shape(slc))
            np.save(os.path.join('/home','jovyan','work',self.dataset_id,names[i]),slc)
            i+=1


SAR_get = Get_image(target_id1)
SAR_get.SAR_data()
print('GET SAR SLC DATA1 AS NUMPY!!')

SAR_get = Get_image(target_id2)
SAR_get.SAR_data()
print('GET SAR SLC DATA2 AS NUMPY!!')

In [ ]:
# 結果の可視化(確認用)
import cv2
import matplotlib.pyplot as plt
from numpy import inf
import numpy as np

Shh = np.load('/home/jovyan/work/'+ target_id1 + '/HH.npy')
Shv = np.load('/home/jovyan/work/'+ target_id1 + '/HV.npy')
Svv = np.load('/home/jovyan/work/'+ target_id1 + '/VV.npy')

sigma = 10*np.log10(Shh) -83.0 -32.0
sigma = np.array(255*(sigma-np.amin(sigma))/(np.amax(sigma)-np.amin(sigma)),dtype="uint8")
HH = sigma

sigma = 10*np.log10(Shv) -83.0 -32.0
sigma = np.array(255*(sigma-np.amin(sigma))/(np.amax(sigma)-np.amin(sigma)),dtype="uint8")
HV = sigma

sigma = 10*np.log10(Svv) -83.0 -32.0
sigma = np.array(255*(sigma-np.amin(sigma))/(np.amax(sigma)-np.amin(sigma)),dtype="uint8")
VV = sigma


img=np.zeros((2000, 2000, 3), np.uint8)
img[:,:,0]=HH
img[:,:,1]=HV
img[:,:,2]=VV

# ルックアップテーブルの生成(輝度の調整)
min_table = 70
max_table = 255
diff_table = max_table - min_table
look_up_table = np.arange(256, dtype = 'uint8')
for i in range(0, min_table):
    look_up_table[i] = 0
for i in range(min_table, max_table):
    look_up_table[i] = 255 * (i - min_table) / diff_table
for i in range(max_table, 255):
    look_up_table[i] = 255

img = cv2.LUT(img, look_up_table)

plt.figure()
plt.imshow(cv2.flip(img,1))
plt.imsave('img1.png', cv2.flip(img,1))